# Install and Import Dependencies

In [9]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('AiFit', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [10]:
# Initialize mediapipe pose class.
mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils

In [11]:
def detectPose(image_pose, pose, draw=False, display=False):
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    resultant = pose.process(image_in_RGB)
    marked_img = np.zeros(image_pose.shape)
    if resultant.pose_landmarks and draw:    
        mp_drawing.draw_landmarks(image=marked_img, landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=6, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=3, circle_radius=2))
        mp_drawing.draw_landmarks(image=image_pose, landmark_list=resultant.pose_landmarks,
                              connections=mp_pose.POSE_CONNECTIONS,
                              landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                           thickness=6, circle_radius=3),
                              connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                           thickness=3, circle_radius=2))

    if display:
            plt.figure(figsize=[5,5])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            plt.subplot(122);plt.imshow(marked_img[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');

    else:
        return image_pose, marked_img

# Load Model

In [13]:
labels = ['downdog', 'goddess', 'plank', 'tree', 'warrior2', 'Invalid']

In [14]:
model = load_model('aifit_conv2d.h5')

In [15]:
model.summary()

Model: "aifitconv_2d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 256, 256, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 256, 256, 3)      0         
 tion)                                                           
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 128, 128, 16)     

In [16]:
# image_path = 'DATASET/TRAIN/downdog/00000128.jpg'
# output = cv2.imread(image_path)
# image_pose, marked_img = detectPose(output, pose_video, draw=True, display=False)
# input_img = cv2.resize(marked_img,(256,256))
# input_img=np.asarray(input_img)
# x, y, z = input_img.shape
# pos_index=model.predict(input_img.reshape(1, x, y, z))
# accuracy = []
# for i in range(0, len(list(pos_index[0]))):
#     accuracy.append(pos_index[0][i]*100)
# print('accuracy', max(accuracy))
# detectPose(output, pose_video, draw=True, display=True)

# Make Detections

In [9]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image_pose, marked_img = detectPose(frame, pose_video, draw=True, display=False)
        cv2.imshow('AiFit', image_pose)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [10]:
mp_drawing.DrawingSpec??

In [8]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
y_class = labels[len(labels)-1]
accuracy = [0]
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image_pose, marked_img = detectPose(frame, pose_video, draw=True, display=False)
        if not np.all((marked_img == 0)):
            accuracy = []
            input_img = cv2.resize(marked_img,(256,256))
            input_img=np.asarray(input_img)
            x, y, z = input_img.shape
            input_img = input_img.reshape(1, x, y, z)
            pos_index=model.predict(input_img)
            for i in range(0, len(list(pos_index[0]))):
                accuracy.append(pos_index[0][i]*100)
            pos_index=np.argmax(pos_index,axis=1)[0]
            y_class = labels[pos_index]
            if round(max(accuracy), 1) < 50:
                y_class = labels[len(labels)-1]
                accuracy = [0]
            

        # Setup status box
        cv2.rectangle(image_pose, (0,0), (225,73), (245,117,16), -1)
        # data
        cv2.putText(image_pose, y_class, (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image_pose, str(round(max(accuracy), 1)) +'%', 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('AiFit', image_pose)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 68ms/step


1/1 [==============================] - 0s 36ms/step
